In [1]:
import numpy as np

In [2]:
weathers = ['sunny', 'cloudy', 'rainy']

## 2(a).

In [3]:
def weather_probs(weather_start: str, trans_matrix: np.ndarray, day: int):
    assert day > 0
    
    probs = (np.array(weathers) == weather_start).astype(int)
    for i in range(day-1):
        probs = np.matmul(probs, trans_matrix)
        
    return probs

In [4]:
day1 = 'sunny'
P1 = np.array([[.8, .2, .0], [.4, .4, .2], [.2, .6, .2]])

In [5]:
print(f"P(Day2=cloudy): {weather_probs(day1, P1, 2)[weathers.index('cloudy')]:.2f}")
print(f"P(Day3=cloudy): {weather_probs(day1, P1, 3)[weathers.index('cloudy')]:.2f}")
print(f"P(Day4=rainy): {weather_probs(day1, P1, 4)[weathers.index('rainy')]:.2f}")

P(Day2=cloudy): 0.20
P(Day3=cloudy): 0.24
P(Day4=rainy): 0.06


## 2(b).

In [6]:
def random_weather_seq(init_probs: np.ndarray, trans_matrix: np.ndarray, n_days: int):
    assert n_days > 0
    
    seq = list()
    next_probs = init_probs
    for i in range(n_days):
        seq.append(np.random.choice(weathers, p=next_probs))
        next_probs = np.matmul(next_probs, trans_matrix)
        
        
    return seq

In [7]:
random_weather_seq(np.full(3, 1/3), P1, 7)

['cloudy', 'cloudy', 'sunny', 'rainy', 'sunny', 'cloudy', 'cloudy']

## 2(c).

In [8]:
def stationary_distribution(init_probs, trans_matrix, n_samples=10000):
    probs = init_probs
    step = 0
    while True:
        new_probs = np.matmul(probs, trans_matrix)
        if np.allclose(probs, new_probs, rtol=1e-16, atol=1e-16):
            print(f'Found stationary distribution at {step}th steps.')
            return probs
        
        probs = new_probs
        step += 1

In [9]:
pi = stationary_distribution(np.full(3, 1/3), P1)
pi

Found stationary distribution at 49th steps.


array([0.64285714, 0.28571429, 0.07142857])

## 2(d).

* Definition of **Stationary Distribution (Discrete ver.)**  
  Let $\pi$ be a probability vector and let $P$ be a state transition matrix.  
  If $\pi = \pi P$, then $\pi$ is called the **stationary distribution** of the state transition matrix $P$.
* Given the state transition matrix
    $$
    P = 
    \begin{pmatrix}
    0.8 & 0.2 & 0 \\
    0.4 & 0.4 & 0.2 \\
    0.2 & 0.6 & 0.2
    \end{pmatrix}
    $$
  Let $\pi = [\pi_0, \pi_1, \pi_2]$. We can write $\pi = \pi P$ as 
  $$
  \begin{pmatrix}
  \pi_0 & \pi_1 & \pi_2  
  \end{pmatrix} = 
  \begin{pmatrix}
  \pi_0 & \pi_1 & \pi_2  
  \end{pmatrix}
  \begin{pmatrix}
  0.8 & 0.2 & 0 \\
  0.4 & 0.4 & 0.2 \\
  0.2 & 0.6 & 0.2
  \end{pmatrix}
  $$
  Then we can derive that
  $$
  \begin{array}{rcl}
  \pi_0 & = & 0.8\pi_0 + 0.4\pi_1 + 0.2\pi_2 \\
  \pi_1 & = & 0.2\pi_0 + 0.4\pi_1 + 0.6\pi_2 \\
  \pi_2 & = & 0.2\pi_1 + 0.2\pi_2
  \end{array}
  $$
  
  By solving this equation system above, we can find the solution is in this form $(9t, 4t, t)$ for any $t$.  
  Let $\displaystyle t = \frac{1}{9+4+1} = \frac{1}{14}$. Then the **stationary distribution** $\displaystyle \pi = (\frac{9}{14}, \frac{2}{7}, \frac{1}{14})$. 

## 2(e).

* By definition, entropy $\displaystyle H(X) = \mathbf{E}[-\log p(X)] = -\sum_{x \in X}p(x)\log p(x)$
* Thus the entropy of the stationary distribution $\pi$ will be
  $$
  \displaystyle \mathbf{E}[-\log\pi] = -\{\frac{9}{14}\log(\frac{9}{14}) + \frac{2}{7}\log(\frac{2}{7}) + \frac{1}{14}\log(\frac{1}{14})\} = 1.1981...
  $$
* The next two cells verified our answer.

In [10]:
# the algorithm which is from scratch
import math

def entropy(probs):
    return -np.sum([p*math.log2(p) for p in probs])

entropy(pi)

1.1981174211304029

In [11]:
# the algorithm provided by SciPy
from scipy.stats import entropy

entropy(pi, base=2)

1.1981174211304035

## 2(f).
* **Bayes' Rule:** $$\displaystyle P(X|Y) = \frac{P(X)P(Y|X)}{P(Y)}$$
* Let r.v. $X$ be yesterday's weather and let r.v. $Y$ be today's weather. Want to find the probability table of $P(X|Y)$.
* Note that $X\sim\pi$ and $Y\sim\pi$. And $\pi$ is given from **2(d).**
* Then the probability table will be  
| WEATHER | Sunny  | Cloudy | Rainy  |  
|---------|--------|--------|--------|  
| Sunny   | 0.8    | 0.1778 | 0.0222 |  
| Cloudy  | 0.45   | 0.4    | 0.15   |  
| Rainy   | 0      | 0.8    | 0.2    |  

## 2(g).

* Answer: **No, it will not violate the Markov property**
* Definition of **Markov property**:
  $$P[X_{t+1} = x | X_1=x_1, X_2=x_2, ..., X_t=x_t] = P[X_{t+1} = x | X_t=x_t], \forall t>0$$
* Let $P_i$ be the state transition matrix for $T_i \leq t < T_{i+1}$.
* Let's discuss case by case:
    * case 1: $T_0 \leq t < T_1$
        * Trivial. The state transition matrix is always $P_0$ so it satisfies Markov property.
    * case 2: $T_1 \leq t < T_2$
        * Note that the state transition matrix $P_1$ describes $P[X_{t+1} = x | X_t=x_t]$.
        * No matter what the previous state transitions are, the state transition matrix is always $P_1$ in this time interval.
        * Thus it satisfies Markov property in this time interval.
    * case 3: $T_2 \leq t < T_3$ (similar as case 2.)
    * case 4: $T_3 \leq t < T_4$ (similar as case 2.)
    * Thus it satisfies Markov property for any $t$

## 3(a).

* Let r.v. $X$ be the weather detected by the sensor and let r.v. $Y$ be the actual weather.
* Consider Day2 *(cloudy)*:
    $$
    \begin{aligned}
    P[Y_2=sunny|X_2=cloudy]  & = \frac{P[Y_2=sunny]P[X_2=cloudy|Y_2=sunny]}{\rho} \\
                             & = \frac{0.32}{\rho} = 0.7 \\
                             \\
    P[Y_2=cloudy|X_2=cloudy] & = \frac{P[Y_2=cloudy]P[X_2=cloudy|Y_2=cloudy]}{\rho} \\
                             & = \frac{0.14}{\rho} = 0.3 \\
                             \\
    P[Y_2=rainy|X_2=cloudy]  & = \frac{P[Y_2=rainy]P[X_2=cloudy|Y_2=rainy]}{\rho} \\
                             & = 0
    \end{aligned}
    $$
    
* Consider Day3 *(cloudy)*:
    $$
    \begin{aligned}
    P[Y_3=sunny|X_2=cloudy, X_3=cloudy]  & = \frac{P[Y_3=sunny|X_2=cloudy]P[X_3=cloudy|X_2=cloudy, Y_3=sunny]}{\rho} \\
                                         & = \frac{0.271}{\rho} = 0.6 \\
                                         \\
    P[Y_3=cloudy|X_2=cloudy, X_3=cloudy] & = \frac{P[Y_3=cloudy|X_2=cloudy]P[X_3=cloudy|X_2=cloudy, Y_3=cloudy]}{\rho} \\
                                         & = \frac{0.183}{\rho} = 0.4 \\
                                         \\
    P[Y_3=rainy|X_2=cloudy, X_3=cloudy]  & = \frac{P[Y_3=rainy|X_2=cloudy]P[X_3=cloudy|X_2=cloudy, Y_3=rainy]}{\rho} \\
                                         & = 0
    \end{aligned}
    $$
    
* Consider Day4 *(rainy)*:
    $$
    \begin{aligned}
    P[Y_4=sunny|X_{2:4}=(c, c, r)]  & = \frac{P[Y_4=sunny|X_{2:3}=(c, c)]P[X_4=rainy|X_{2:3}=(c, c), Y_4=sunny]}{\rho} \\
                                         & = 0 \\
                                         \\
    P[Y_4=cloudy|X_{2:4}=(c, c, r)] & = \frac{P[Y_4=cloudy|X_{2:3}=(c, c)]P[X_4=rainy|X_{2:3}=(c, c), Y_4=cloudy]}{\rho} \\
                                         & = 0 \\
                                         \\
    P[Y_4=rainy|X_{2:4}=(c, c, r)]  & = \frac{P[Y_4=rainy|X_{2:3}=(c, c)]P[X_4=rainy|X_{2:3}=(c, c), Y_4=rainy]}{\rho} \\
                                         & = 1
    \end{aligned}
    $$
    
* Consider Day5 *(sunny)*:
    $$
    \begin{aligned}
    P[Y_5=sunny|X_{2:5}=(c, c, r, s)]  & = \frac{P[Y_5=sunny|X_{2:4}=(c, c, r)]P[X_5=sunny|X_{2:4}=(c, c, r), Y_5=sunny]}{\rho} \\
                                         & = \frac{0.271}{\rho} = 0.4 \\
                                         \\
    P[Y_5=cloudy|X_{2:5}=(c, c, r, s)] & = \frac{P[Y_5=cloudy|X_{2:4}=(c, c, r)]P[X_5=sunny|X_{2:4}=(c, c, r), Y_5=cloudy]}{\rho} \\
                                         & = \frac{0.183}{\rho} = 0.6 \\
                                         \\
    P[Y_5=rainy|X_{2:5}=(c, c, r, s)]  & = \frac{P[Y_5=rainy|X_{2:4}=(c, c, r)]P[X_5=sunny|X_{2:4}=(c, c, r), Y_5=rainy]}{\rho} \\
                                         & = 0
    \end{aligned}
    $$
    
* Answer: $P[Y_5=sunny|X_{2:5}=(c, c, r, s)] = 0.4$

## 3(b).
* Let r.v. $X$ be the weather detected by the sensor and let r.v. $Y$ be the actual weather.
* Consider Day2 *(sunny)*:
    $$
    \begin{aligned}
    P[Y_2=sunny|X_2=sunny]  & = \frac{P[Y_2=sunny]P[X_2=sunny|Y_2=sunny]}{\rho} \\
                            & = \frac{0.32}{\rho} = 0.89 \\
                            \\
    P[Y_2=cloudy|X_2=sunny] & = \frac{P[Y_2=cloudy]P[X_2=sunny|Y_2=cloudy]}{\rho} \\
                            & = \frac{0.14}{\rho} = 0.11 \\
                            \\
    P[Y_2=rainy|X_2=sunny]  & = \frac{P[Y_2=rainy]P[X_2=sunny|Y_2=rainy]}{\rho} \\
                            & = 0
    \end{aligned}
    $$
    
* Consider Day3 *(sunny)*:
    $$
    \begin{aligned}
    P[Y_3=sunny|X_2=sunny, X_3=sunny]  & = \frac{P[Y_3=sunny|X_2=sunny]P[X_3=sunny|X_2=sunny, Y_3=sunny]}{\rho} \\
                                       & = \frac{0.271}{\rho} = 0.87 \\
                                       \\
    P[Y_3=cloudy|X_2=sunny, X_3=sunny] & = \frac{P[Y_3=cloudy|X_2=sunny]P[X_3=sunny|X_2=sunny, Y_3=cloudy]}{\rho} \\
                                       & = \frac{0.183}{\rho} = 0.13 \\
                                       \\
    P[Y_3=rainy|X_2=sunny, X_3=sunny]  & = \frac{P[Y_3=rainy|X_2=sunny]P[X_3=sunny|X_2=sunny, Y_3=rainy]}{\rho} \\
                                       & = 0
    \end{aligned}
    $$
    
* Consider Day4 *(rainy)*:
    $$
    \begin{aligned}
    P[Y_4=sunny|X_{2:4}=(s, s, r)]  & = \frac{P[Y_4=sunny|X_{2:3}=(s, s)]P[X_4=rainy|X_{2:3}=(s, s), Y_4=sunny]}{\rho} \\
                                    & = 0 \\
                                    \\
    P[Y_4=cloudy|X_{2:4}=(s, s, r)] & = \frac{P[Y_4=cloudy|X_{2:3}=(s, s)]P[X_4=rainy|X_{2:3}=(s, s), Y_4=cloudy]}{\rho} \\
                                    & = 0 \\
                                    \\
    P[Y_4=rainy|X_{2:4}=(s, s, r)]  & = \frac{P[Y_4=rainy|X_{2:3}=(s, s)]P[X_4=rainy|X_{2:3}=(s, s), Y_4=rainy]}{\rho} \\
                                    & = 1
    \end{aligned}
    $$
    
* Answer: sunny, sunny, rainy

## 3(c).